**PROYECTO CHATVOZ IA CONVERSACIONAL**


**Introducción:**
El desarrollo de un chatbot que interactúa no solo mediante texto, sino también con respuestas auditivas, es una de las mejoras más atractivas en la experiencia del usuario. La tecnología Text-to-Speech (TTS) ha avanzado lo suficiente como para proporcionar respuestas de voz naturales y personalizadas. Al combinar la interacción conversacional del chatbot con la capacidad de generar audio, puedes crear una interfaz más accesible e inmersiva para los usuarios, facilitando el acceso a la información de manera más inclusiva y eficiente.

En este proyecto, hemos integrado un modelo de TTS para que el chatbot no solo responda con texto, sino que también convierta sus respuestas en audio. Esta característica permite que los usuarios no solo lean, sino también escuchen las respuestas del chatbot, mejorando la experiencia y la accesibilidad.

**Objetivos del Proyecto:**
Integrar un sistema de Text-to-Speech (TTS): Implementar un mecanismo en el que el chatbot pueda convertir sus respuestas textuales en audio utilizando tecnologías de conversión de texto a voz, como la biblioteca gTTS.

Crear una interfaz interactiva usando Gradio: Proporcionar una interfaz de usuario en la que el chatbot pueda recibir preguntas, responder con texto, y ofrecer una reproducción en audio de sus respuestas.

Permitir la personalización de la voz: Incluir opciones que permitan a los usuarios ajustar la velocidad de la voz y seleccionar diferentes idiomas para la conversión de texto a voz.

Generar respuestas tanto en texto como en audio: Cada vez que el usuario realice una consulta al chatbot, éste responderá con un texto que se podrá leer y, simultáneamente, con un archivo de audio que se puede reproducir directamente en la interfaz.

**Gestionar el historial de conversación:** Mantener un registro del historial de conversación entre el usuario y el chatbot, con un formato claro y visual, que permita ver las interacciones anteriores.

Facilitar la carga de documentos PDF: Proporcionar la capacidad de cargar y procesar documentos PDF, de los cuales el chatbot puede extraer información para responder preguntas de manera contextual.

Este enfoque ofrece una experiencia de interacción enriquecida, con la ventaja añadida de la accesibilidad auditiva, ideal para usuarios que prefieren o necesitan respuestas por voz en lugar de texto.



In [1]:
!pip install pypdf
!pip install langchain
!pip install langchain-openai
!pip install gtts
!pip install playsound
!pip install python-dotenv
!pip install openai-whisper
!pip install pymupdf pdfplumber PyPDF2 gradio faiss-cpu tiktoken
!pip install -U langchain-community
!pip install openai
!pip install gtts



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/

In [2]:
# Vamos a generar un archivo .py que contenga el código completo que necesitas

from gtts import gTTS
import gradio as gr
import os
import tempfile
import requests
import time
import PyPDF2
from dotenv import load_dotenv


**Cargar las variables de entorno desde el\ archivo .env**

In [82]:
load_dotenv()

False

In [83]:
dotenv_path = '/content/sample_data/Clave.env'  # Reemplaza con la ruta correcta a tu archivo .env
load_dotenv(dotenv_path)

True

In [84]:
api_key = os.getenv("OPENAI_API_KEY")

In [64]:
if not api_key:
    raise ValueError("La clave de API no está definida. Asegúrate de que está configurada correctamente en el archivo .env.")

# Imprimir para verificar que se ha cargado correctamente (opcional)
print(f"Clave API cargada: ")

Clave API cargada: 


 **Funcion para cargar la clave API**

In [65]:
def cargar_clave_api():
    # Obtener la clave de API
    api_key = os.getenv("OPENAI_API_KEY")

    # Verificar si la clave se ha cargado correctamente
    if api_key:
        return "Clave API cargada correctamente"
    else:
        return "Error al cargar la clave API. Verifica el archivo .env."

In [66]:

# Configuración del endpoint
endpoint = "https://aoai-ine.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-15-preview"


 **Función para enviar solicitudes a OpenAI
Envía una solicitud al modelo de OpenAI,pasando el prompt (instrucciones) y la entrada del usuario,
y maneja posibles errores de red (especialmente errores HTTP 429 que indican demasiadas solicitudes).**

In [67]:
def send_request_to_model(prompt, user_input, api_key, endpoint, retries=5):
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key,
    }

    payload = {
        "messages": [
            {"role": "system", "content": prompt},  # Aquí se integra el contenido del PDF
            {"role": "user", "content": user_input}
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800
    }

    attempt = 0
    while attempt < retries:
        try:
            response = requests.post(endpoint, headers=headers, json=payload)
            response.raise_for_status()
            response_json = response.json()
            return response_json['choices'][0]['message']['content']
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                attempt += 1
                wait_time = 5 * (2 ** attempt)
                print(f"HTTP 429: Too Many Requests. Reintentando en {wait_time} segundos...")
                time.sleep(wait_time)
            else:
                raise SystemExit(f"Fallo en la solicitud. Error: {e}")
    raise SystemExit(f"Fallo en la solicitud después de {retries} reintentos.")

 **Función para convertir texto a audio usando gTTS
Convierte una cadena de texto en un archivo de audio utilizando Google Text-to-Speech (gTTS).**

In [68]:
def convertir_texto_a_audio(texto):
    """Convierte un texto a audio utilizando gTTS."""
    tts = gTTS(texto, lang='es')
    audio_path = "output_audio.mp3"
    tts.save(audio_path)
    return audio_path

**Función para extraer texto desde un PDF usando PyPDF2
Extrae texto de un archivo PDF utilizando PyPDF2.
Detalles:
Extracción por página: Extrae el texto página por página.
Control de errores: Verifica si el archivo existe y si es posible extraer texto de las páginas. Si alguna página está vacía, lanza advertencias.**

In [69]:
def extract_text_from_pdf(pdf_path):
    text = ""
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"El archivo PDF no se encuentra en la ruta especificada: {pdf_path}")

    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num, page in enumerate(reader.pages):
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n\n"
                else:
                    print(f"Advertencia: No se pudo extraer texto de la página {page_num + 1}.")
    except Exception as e:
        raise ValueError(f"Error al intentar leer el archivo PDF: {e}")

    if not text.strip():
        raise ValueError("No se pudo extraer texto del PDF o el archivo está vacío.")

    return text


**Divide el texto en fragmentos ("chunks") de tamaño manejable, limitados por el número de caracteres.
Palabras: El texto se divide por palabras y se reorganiza en fragmentos más pequeños basados en el límite de tamaño (max_chunk_size).
**El propósito de esta función es esencial para reducir la longitud de textos largos. Está bien situada en el código y debería ejecutarse después de extraer el texto de los PDFs**

In [70]:
def chunk_text(text, max_chunk_size=1000):
    chunks = []
    words = text.split(" ")
    current_chunk = []
    for word in words:
        if len(" ".join(current_chunk)) + len(word) + 1 <= max_chunk_size:
            current_chunk.append(word)
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

In [71]:
# este codigo lo usamos para un unico PDF
# Cargar y chunkear un único PDF
# Este codigo lo dejamos implementado pero comentado para un archivo pdf
#def load_single_pdf_and_chunk(pdf_path):
 #   text = extract_text_from_pdf(pdf_path)
  #  if text:
   #     print("Texto extraído correctamente del PDF. Aquí hay una muestra:")
    #    print(text[:500])  # Imprimir los primeros 500 caracteres del texto
    #else:
     #   print("No se pudo extraer texto del PDF.")
    #chunks = chunk_text_by_paragraphs(text)
    #return chunks

**Función para cargar y extraer el texto de todos los PDFs en una carpeta
Propósito:
Carga todos los PDFs de un directorio, extrae el texto y los divide en fragmentos ("chunks")**

* **Detalles:
Extracción masiva: Procesa todos los archivos PDF en un directorio.
Integración con extract_text_from_pdf y chunk_text: Combina las funciones anteriores para obtener los fragmentos.#Esta función la usamos para varios pdf***

In [72]:
def load_pdfs_from_directory(directory_path):
    all_chunks = []
    for file_name in os.listdir(directory_path):
        if file_name.endswith(".pdf"):  # Solo procesar archivos PDF
            pdf_path = os.path.join(directory_path, file_name)
            text = extract_text_from_pdf(pdf_path)
            if text:
                print(f"Texto extraído correctamente del archivo {file_name}.")
                # Dividimos el texto en chunks manejables
                chunks = chunk_text(text, max_chunk_size=1000)
                all_chunks.extend(chunks)  # Añadir todos los chunks a la lista
            else:
                print(f"No se pudo extraer texto del archivo {file_name}.")

    if not all_chunks:
        raise ValueError("No se pudo extraer texto de ningún archivo PDF o todos los archivos están vacíos.")

    return all_chunks  # Devolver los chunks en lugar del texto completo

 **Función para encontrar el chunk más relevante basado en la pregunta del usuario
Propósito:
Encuentra el fragmento más relevante en función de la cantidad de palabras comunes entre la pregunta del usuario y los fragmentos de texto.
Detalles:
Búsqueda basada en intersección: Compara las palabras en el fragmento y la pregunta del usuario para calcular el "overlap" (superposición)
y selecciona el fragmento más relevante.
Esta función es esencial para asegurarse de que el modelo reciba el fragmento más relevante antes de responder.**

In [73]:
def find_relevant_chunk(chunks, user_input):
    """Encuentra el chunk más relevante en función de la pregunta del usuario."""
    relevant_chunk = ""
    max_overlap = 0
    user_words = set(user_input.lower().split())

    # Comparamos la cantidad de palabras comunes entre el chunk y la pregunta
    for chunk in chunks:
        chunk_words = set(chunk.lower().split())
        overlap = len(user_words.intersection(chunk_words))
        if overlap > max_overlap:
            max_overlap = overlap
            relevant_chunk = chunk

    return relevant_chunk

 **Función para manejar la conversación con el chatbot
Propósito:
Maneja la conversación entre el usuario y el chatbot, buscando el fragmento relevante,
enviando la solicitud al modelo y devolviendo la respuesta en texto y audio (opcional).**

**Detalles:
Combinación de funciones: Integra todas las funciones anteriores para buscar el fragmento relevante,
interactuar con el modelo y generar una respuesta.
Audio opcional: Si se solicita, también convierte la respuesta en formato de audio.
Sugerencia:
Es el punto central que conecta todas las demás funcionalidades del chatbot**

In [74]:
def chatbot_conversation(chunks, user_input, api_key, endpoint, respuesta_con_audio=False):
    # Encontrar el chunk más relevante
    relevant_chunk = find_relevant_chunk(chunks, user_input)

    if relevant_chunk:
        prompt = f"Basado en el siguiente texto extraído de archivos PDF:\n\n{relevant_chunk}\n\nResponde a la pregunta del usuario:"
    else:
        prompt = "No se pudo encontrar información relevante en los archivos PDF."

    try:
        response = send_request_to_model(prompt, user_input, api_key, endpoint)
        audio_file = None
        if respuesta_con_audio:
            # Convertir la respuesta a audio
            audio_file = convertir_texto_a_audio(response)
        return response, audio_file
    except Exception as e:
        print(f"Error en la conversación con el chatbot: {e}")
        return "Error en la conversación con el modelo.", None

In [75]:
# Función para formatear el historial del chat en HTML
def format_history(chat_history):
    formatted_history = ""
    for entry in chat_history:
        if entry["role"] == "user":
            formatted_history += f"<div class='user'><i class='fas fa-user-circle'></i> <strong>Usuario:</strong> {entry['content']}</div>"
        else:
            formatted_history += f"<div class='bot'><i class='fas fa-robot'></i> <strong>Bot:</strong> {entry['content']}</div>"
    return formatted_history

In [80]:
from datetime import datetime

# Función para obtener la fecha y la hora actual
def obtener_fecha_hora():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [76]:
# Función para formatear el historial del chat en HTML con burbujas de color
def format_history(chat_history):
    formatted_history = ""
    for entry in chat_history:
        timestamp = obtener_fecha_hora()  # Añadimos la marca de tiempo para cada mensaje
        if entry["role"] == "user":
            formatted_history += f"""
            <div style="background-color: #d4e6f1; color: #000; border-radius: 10px; padding: 10px; margin-bottom: 10px; width: fit-content;">
                <strong>🧑 Usuario:</strong> {entry['content']}
                <br><small>{timestamp}</small>
            </div>
            """
        else:
            formatted_history += f"""
            <div style="background-color: #b8e994; color: #000; border-radius: 10px; padding: 10px; margin-bottom: 10px; width: fit-content;">
                <strong>🤖 Bot:</strong> {entry['content']}
                <br><small>{timestamp}</small>
            </div>
            """
    return formatted_history

 **Interfaz del chatbot
Propósito:
Crea la interfaz gráfica para interactuar con el chatbot usando Gradio.
Detalles:
Gradio: Genera la interfaz donde el usuario puede hacer preguntas al chatbot, ver el historial de chat y recibir respuestas en texto o audio.
Interacción con el chatbot: Llama a las funciones necesarias para procesar las solicitudes del usuario y mostrar las respuestas.
Utilizamos Gradio para manejar las interacciones con el usuario.**

In [ ]:
def chatbot_ui():
    try:
        # Directorio donde se encuentran los archivos PDF
        directory_path = "/content/sample_data/EncuestasECH"  # Cambia esto a la ruta de tu carpeta de PDFs

        # Cargar y chunkear todos los PDFs desde la carpeta especificada
        chunks = load_pdfs_from_directory(directory_path)

        print(f"Se han cargado {len(chunks)} chunks de los archivos PDF.")

        chat_history = []  # Inicializamos el historial vacío

        with gr.Blocks() as demo:
            with gr.Row():
                with gr.Column(scale=4):
                    gr.Markdown("# CHATBOT BASADO EN MÚLTIPLES PDFs")

                    # Checkbox para la respuesta en audio
                    respuesta_con_audio = gr.Checkbox(label="¿Deseas que el chatbot te responda con audio?", value=False)

                    # Campo de entrada de texto para preguntas al chatbot
                    user_input = gr.Textbox(label="Pregunta al chatbot", placeholder="Escribe tu pregunta aquí...")

                    # Botón para enviar la pregunta
                    send_button = gr.Button("Enviar")

                    # Campo para mostrar la respuesta del chatbot
                    chat_output = gr.Textbox(label="Respuesta del chatbot", interactive=False)

                    # Campo para mostrar el audio de la respuesta
                    audio_output = gr.Audio(label="Respuesta en Audio", interactive=False)

                with gr.Column(scale=2):
                    with gr.Row():
                        # Botón para limpiar el historial
                        clear_button = gr.Button("Limpiar historial")

                    gr.Markdown("## Historial del Chat")

                    # Campo para mostrar el historial del chat
                    history_output = gr.HTML(value="""
                        <div style="border: 2px solid #ccc; border-radius: 15px; padding: 15px; background-color: #f9f9f9; height: 400px; width: 100%; overflow-y: auto;" id="chat_container">
                        </div>
                    """)

            # Función para interactuar con el chatbot
            def interact(input_text, con_audio):
                nonlocal chat_history, chunks
                answer, audio_file = chatbot_conversation(chunks, input_text, api_key, endpoint, con_audio)
                timestamp = obtener_fecha_hora()

                # Actualizar el historial con la nueva interacción
                chat_history.append({"role": "user", "content": input_text})  # Añadir entrada del usuario
                chat_history.append({"role": "bot", "content": answer})  # Añadir respuesta del bot

                # Formatear el historial del chat
                formatted_history = format_history(chat_history)

                return answer, formatted_history, audio_file if con_audio else None

            # Vincular las funciones a los eventos
            send_button.click(interact, inputs=[user_input, respuesta_con_audio], outputs=[chat_output, history_output, audio_output])

        demo.launch(share=True, debug=True)

    except Exception as e:
        print(f"Error creando la interfaz del chatbot: {e}")

In [79]:
# Ejecutar la interfaz
if __name__ == "__main__":
    chatbot_ui()


Error creando la interfaz del chatbot: [Errno 2] No such file or directory: '/content/sample_data/EncuestasECH'
